In [1]:
import pyspark
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import to_timestamp, col, shuffle, rand
from pyspark.sql.types import LongType, DoubleType, StringType, TimestampType, StructType, StructField, DateType, FloatType, IntegerType, ArrayType
import datetime

In [2]:
sc = SparkContext("local")
spark = SparkSession(sc)

21/08/04 13:40:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
test_user_schema = StructType([
  StructField('SM_USERNAME', StringType()),
  StructField('SM_RESOURCE', StringType()),
  StructField('SM_TIMESTAMP_TEMP', StringType())
])

ans_schema = StructType([
  StructField('SM_USERNAME', StringType()),
  StructField('window_temp', StructType([  
    StructField('start', StringType()),
    StructField('end', StringType())
  ])),
  StructField('SM_RESOURCE', ArrayType(StringType()))
])


In [14]:
def process_test_data(test_data):
    test_df = spark.createDataFrame(test_data, schema=test_user_schema)
    test_df = test_df.withColumn('SM_TIMESTAMP', col('SM_TIMESTAMP_TEMP').cast('timestamp'))
    test_df = test_df.drop('SM_TIMESTAMP_TEMP')
    return test_df

def process_expected_data(expected_data):
    ans_df = spark.createDataFrame(expected_data, schema=ans_schema)
    ans_df = ans_df.withColumn('window', col('window_temp').cast(StructType([StructField('start', TimestampType()),StructField('end', TimestampType())])))
    ans_df = ans_df.drop('window_temp')
    ans_df = ans_df.select('SM_USERNAME', 'window', 'SM_RESOURCE')
    return ans_df

In [15]:
user_one_window_10 = [
  
  ('User_A', 'Resource_A', '2018-02-01T00:00:00.000+0000'),
  ('User_A', 'Resource_B', '2018-02-01T00:01:00.000+0000'),
  ('User_A', 'Resource_C', '2018-02-01T00:02:00.000+0000'),
  ('User_B', 'Resource_A', '2018-02-01T00:01:00.000+0000'),
  ('User_B', 'Resource_B', '2018-02-01T00:02:00.000+0000'),
  ('User_B', 'Resource_C', '2018-02-01T00:03:00.000+0000'),
  ('User_B', 'Resource_D', '2018-02-01T00:04:00.000+0000'),
  ('User_B', 'Resource_E', '2018-02-01T00:05:00.000+0000'),
  ('User_B', 'Resource_B', '2018-02-01T00:06:00.000+0000'),
  ('User_B', 'Resource_D', '2018-02-01T00:07:00.000+0000'),
  ('User_B', 'Resource_A', '2018-02-01T00:08:00.000+0000'),
  ('User_B', 'Resource_E', '2018-02-01T00:09:00.000+0000'),
  ('User_C', 'Resource_A', '2018-02-01T00:02:00.000+0000'),
  ('User_C', 'Resource_C', '2018-02-01T00:05:00.000+0000'),
  ('User_C', 'Resource_B', '2018-02-01T00:05:01.000+0000'),
  ('User_C', 'Resource_A', '2018-02-01T00:05:02.000+0000'),
  ('User_C', 'Resource_D', '2018-02-01T00:07:00.000+0000'),
  ('User_D', 'Resource_A', '2018-02-01T00:06:00.000+0000'),
  ('User_D', 'Resource_B', '2018-02-01T00:08:00.000+0000'),
  ('User_D', 'Resource_C', '2018-02-01T00:08:01.000+0000'),
  ('User_D', 'Resource_D', '2018-02-01T00:08:04.000+0000'),
  ('User_D', 'Resource_E', '2018-02-01T00:09:00.000+0000'),
  ('User_D', 'Resource_F', '2018-02-01T00:09:03.000+0000'),
  ('User_D', 'Resource_G', '2018-02-01T00:11:00.000+0000'),
  ('User_D', 'Resource_H', '2018-02-01T00:12:00.000+0000'),
  ('User_D', 'Resource_I', '2018-02-01T00:13:00.000+0000'),
  ('User_D', 'Resource_J', '2018-02-01T00:14:00.000+0000'),
  ('User_D', 'Resource_B', '2018-02-01T00:14:01.000+0000'),
  ('User_D', 'Resource_C', '2018-02-01T00:14:03.000+0000'),
  ('User_D', 'Resource_A', '2018-02-01T00:14:08.000+0000'),
  ('User_D', 'Resource_B', '2018-02-01T00:14:09.000+0000'),
  ('User_D', 'Resource_C', '2018-02-01T00:14:40.000+0000'),
  ('User_D', 'Resource_A', '2018-02-01T00:14:50.000+0000'),
  ('User_E', 'Resource_A', '2018-02-01T00:01:00.000+0000'),
  ('User_E', 'Resource_C', '2018-02-01T00:09:00.000+0000'),

]

user_one_window_10 = process_test_data(user_one_window_10)
user_one_window_10.show()

+-----------+-----------+-------------------+
|SM_USERNAME|SM_RESOURCE|       SM_TIMESTAMP|
+-----------+-----------+-------------------+
|     User_A| Resource_A|2018-02-01 00:00:00|
|     User_A| Resource_B|2018-02-01 00:01:00|
|     User_A| Resource_C|2018-02-01 00:02:00|
|     User_B| Resource_A|2018-02-01 00:01:00|
|     User_B| Resource_B|2018-02-01 00:02:00|
|     User_B| Resource_C|2018-02-01 00:03:00|
|     User_B| Resource_D|2018-02-01 00:04:00|
|     User_B| Resource_E|2018-02-01 00:05:00|
|     User_B| Resource_B|2018-02-01 00:06:00|
|     User_B| Resource_D|2018-02-01 00:07:00|
|     User_B| Resource_A|2018-02-01 00:08:00|
|     User_B| Resource_E|2018-02-01 00:09:00|
|     User_C| Resource_A|2018-02-01 00:02:00|
|     User_C| Resource_C|2018-02-01 00:05:00|
|     User_C| Resource_B|2018-02-01 00:05:01|
|     User_C| Resource_A|2018-02-01 00:05:02|
|     User_C| Resource_D|2018-02-01 00:07:00|
|     User_D| Resource_A|2018-02-01 00:06:00|
|     User_D| Resource_B|2018-02-0

In [16]:
expected_user_one_window_10 = [
  
  ('User_D', {"start": "2018-02-01T00:00:00.000+0000", "end": "2018-02-01T00:15:00.000+0000"}, ["Resource_A", "Resource_B", "Resource_C", "Resource_D", "Resource_E", "Resource_F", "Resource_G", "Resource_H", \
                                                                                                "Resource_I", "Resource_J"]),
  ('User_C', {"start": "2018-02-01T00:00:00.000+0000", "end": "2018-02-01T00:15:00.000+0000"}, ["Resource_A", "Resource_C", "Resource_B", "Resource_A", "Resource_D"]),
  ('User_A', {"start": "2018-02-01T00:00:00.000+0000", "end": "2018-02-01T00:15:00.000+0000"}, ["Resource_A", "Resource_B", "Resource_C"]),
  ('User_B', {"start": "2018-02-01T00:00:00.000+0000", "end": "2018-02-01T00:15:00.000+0000"}, ["Resource_A", "Resource_B", "Resource_C", "Resource_D", "Resource_E", "Resource_B", "Resource_D", "Resource_A", "Resource_E"]),  
  ('User_E', {"start": "2018-02-01T00:00:00.000+0000", "end": "2018-02-01T00:15:00.000+0000"}, ["Resource_A", "Resource_C"]),
]

expected_user_one_window_10 = process_expected_data(expected_user_one_window_10)
expected_user_one_window_10.show()

+-----------+--------------------+--------------------+
|SM_USERNAME|              window|         SM_RESOURCE|
+-----------+--------------------+--------------------+
|     User_D|{2018-02-01 00:00...|[Resource_A, Reso...|
|     User_C|{2018-02-01 00:00...|[Resource_A, Reso...|
|     User_A|{2018-02-01 00:00...|[Resource_A, Reso...|
|     User_B|{2018-02-01 00:00...|[Resource_A, Reso...|
|     User_E|{2018-02-01 00:00...|[Resource_A, Reso...|
+-----------+--------------------+--------------------+



In [17]:
user_multiple_window_5 = [
  
  ('User_A', 'Resource_A', '2018-02-01T00:00:00.000+0000'),
  ('User_A', 'Resource_B', '2018-02-01T00:01:00.000+0000'),
  ('User_A', 'Resource_C', '2018-02-01T02:00:00.000+0000'),
  ('User_B', 'Resource_A', '2018-02-01T00:01:00.000+0000'),
  ('User_B', 'Resource_B', '2018-02-01T00:02:00.000+0000'),
  ('User_B', 'Resource_C', '2018-02-01T00:03:00.000+0000'),
  ('User_B', 'Resource_D', '2018-02-01T00:04:00.000+0000'),
  ('User_B', 'Resource_E', '2018-02-01T00:05:00.000+0000'),
  ('User_B', 'Resource_B', '2018-02-01T00:06:00.000+0000'),
  ('User_B', 'Resource_D', '2018-02-01T00:20:00.000+0000'),
  ('User_B', 'Resource_A', '2018-02-01T00:20:00.000+0000'),
  ('User_B', 'Resource_E', '2018-02-01T00:20:00.000+0000'),
  ('User_C', 'Resource_A', '2018-02-01T00:02:00.000+0000'),
  ('User_C', 'Resource_C', '2018-02-01T00:05:00.000+0000'),
  ('User_C', 'Resource_B', '2018-02-01T00:20:01.000+0000'),
  ('User_C', 'Resource_A', '2018-02-01T00:20:02.000+0000'),
  ('User_C', 'Resource_D', '2018-02-01T00:20:00.000+0000'),
  ('User_D', 'Resource_A', '2018-02-01T00:06:00.000+0000'),
  ('User_D', 'Resource_B', '2018-02-01T00:08:00.000+0000'),
  ('User_D', 'Resource_C', '2018-02-01T00:08:01.000+0000'),
  ('User_D', 'Resource_D', '2018-02-01T00:08:04.000+0000'),
  ('User_D', 'Resource_E', '2018-02-01T00:09:00.000+0000'),
  ('User_D', 'Resource_F', '2018-02-01T00:09:03.000+0000'),
  ('User_D', 'Resource_G', '2018-02-01T00:11:00.000+0000'),
  ('User_D', 'Resource_H', '2018-02-01T00:12:00.000+0000'),
  ('User_D', 'Resource_I', '2018-02-01T00:13:00.000+0000'),
  ('User_D', 'Resource_J', '2018-02-01T01:14:00.000+0000'),
  ('User_D', 'Resource_B', '2018-02-01T01:14:01.000+0000'),
  ('User_D', 'Resource_C', '2018-02-01T01:14:03.000+0000'),
  ('User_D', 'Resource_A', '2018-02-01T01:14:08.000+0000'),
  ('User_D', 'Resource_B', '2018-02-01T01:14:09.000+0000'),
  ('User_D', 'Resource_C', '2018-02-01T01:14:40.000+0000'),
  ('User_D', 'Resource_A', '2018-02-01T01:14:50.000+0000'),
  ('User_E', 'Resource_A', '2018-02-01T00:01:00.000+0000'),
  ('User_E', 'Resource_C', '2018-02-01T01:09:00.000+0000'),

]

user_multiple_window_5 = process_test_data(user_multiple_window_5)
user_multiple_window_5.show()

+-----------+-----------+-------------------+
|SM_USERNAME|SM_RESOURCE|       SM_TIMESTAMP|
+-----------+-----------+-------------------+
|     User_A| Resource_A|2018-02-01 00:00:00|
|     User_A| Resource_B|2018-02-01 00:01:00|
|     User_A| Resource_C|2018-02-01 02:00:00|
|     User_B| Resource_A|2018-02-01 00:01:00|
|     User_B| Resource_B|2018-02-01 00:02:00|
|     User_B| Resource_C|2018-02-01 00:03:00|
|     User_B| Resource_D|2018-02-01 00:04:00|
|     User_B| Resource_E|2018-02-01 00:05:00|
|     User_B| Resource_B|2018-02-01 00:06:00|
|     User_B| Resource_D|2018-02-01 00:20:00|
|     User_B| Resource_A|2018-02-01 00:20:00|
|     User_B| Resource_E|2018-02-01 00:20:00|
|     User_C| Resource_A|2018-02-01 00:02:00|
|     User_C| Resource_C|2018-02-01 00:05:00|
|     User_C| Resource_B|2018-02-01 00:20:01|
|     User_C| Resource_A|2018-02-01 00:20:02|
|     User_C| Resource_D|2018-02-01 00:20:00|
|     User_D| Resource_A|2018-02-01 00:06:00|
|     User_D| Resource_B|2018-02-0

In [18]:
expected_user_multiple_window_5 = [
  
  ('User_A', {"start": "2018-02-01T00:00:00.000+0000", "end": "2018-02-01T00:15:00.000+0000"}, ["Resource_A", "Resource_B"]),
  ('User_A', {"start": "2018-02-01T02:00:00.000+0000", "end": "2018-02-01T02:15:00.000+0000"}, ["Resource_C"]),
  ('User_B', {"start": "2018-02-01T00:00:00.000+0000", "end": "2018-02-01T00:15:00.000+0000"}, ["Resource_A", "Resource_B", "Resource_C", "Resource_D", "Resource_E"]),
  ('User_B', {"start": "2018-02-01T00:15:00.000+0000", "end": "2018-02-01T00:30:00.000+0000"}, ["Resource_D", "Resource_A", "Resource_E"]),  
  ('User_C', {"start": "2018-02-01T00:00:00.000+0000", "end": "2018-02-01T00:15:00.000+0000"}, ["Resource_A", "Resource_C"]),
  ('User_C', {"start": "2018-02-01T00:15:00.000+0000", "end": "2018-02-01T00:30:00.000+0000"}, ["Resource_D", "Resource_B", "Resource_A"]),
  ('User_D', {"start": "2018-02-01T00:00:00.000+0000", "end": "2018-02-01T00:15:00.000+0000"}, ["Resource_A", "Resource_B", "Resource_C","Resource_D", "Resource_E"]),
  ('User_D', {"start": "2018-02-01T01:00:00.000+0000", "end": "2018-02-01T01:15:00.000+0000"}, ["Resource_J", "Resource_B", "Resource_C","Resource_A", "Resource_B"]), 
  ('User_E', {"start": "2018-02-01T00:00:00.000+0000", "end": "2018-02-01T00:15:00.000+0000"}, ["Resource_A"]), 
  ('User_E', {"start": "2018-02-01T01:00:00.000+0000", "end": "2018-02-01T01:15:00.000+0000"}, ["Resource_C"])
 
]

expected_user_multiple_window_5 = process_expected_data(expected_user_multiple_window_5)
expected_user_multiple_window_5.show()

+-----------+--------------------+--------------------+
|SM_USERNAME|              window|         SM_RESOURCE|
+-----------+--------------------+--------------------+
|     User_A|{2018-02-01 00:00...|[Resource_A, Reso...|
|     User_A|{2018-02-01 02:00...|        [Resource_C]|
|     User_B|{2018-02-01 00:00...|[Resource_A, Reso...|
|     User_B|{2018-02-01 00:15...|[Resource_D, Reso...|
|     User_C|{2018-02-01 00:00...|[Resource_A, Reso...|
|     User_C|{2018-02-01 00:15...|[Resource_D, Reso...|
|     User_D|{2018-02-01 00:00...|[Resource_A, Reso...|
|     User_D|{2018-02-01 01:00...|[Resource_J, Reso...|
|     User_E|{2018-02-01 00:00...|        [Resource_A]|
|     User_E|{2018-02-01 01:00...|        [Resource_C]|
+-----------+--------------------+--------------------+



In [19]:
user_duplicate_resources = [
  
  ('User_A', 'Resource_A', '2018-02-01T00:00:00.000+0000'),
  ('User_A', 'Resource_B', '2018-02-01T00:01:00.000+0000'),
  ('User_A', 'Resource_B', '2018-02-01T00:02:00.000+0000'),
  ('User_B', 'Resource_A', '2018-02-01T00:01:00.000+0000'),
  ('User_B', 'Resource_B', '2018-02-01T00:02:00.000+0000'),
  ('User_B', 'Resource_C', '2018-02-01T00:03:00.000+0000'),
  ('User_B', 'Resource_C', '2018-02-01T00:04:00.000+0000'),
  ('User_B', 'Resource_C', '2018-02-01T00:05:00.000+0000'),
  ('User_B', 'Resource_B', '2018-02-01T00:06:00.000+0000'),
  ('User_B', 'Resource_D', '2018-02-01T00:07:00.000+0000'),
  ('User_B', 'Resource_A', '2018-02-01T00:08:00.000+0000'),
  ('User_B', 'Resource_E', '2018-02-01T00:09:00.000+0000'),
  ('User_C', 'Resource_A', '2018-02-01T00:02:00.000+0000'),
  ('User_C', 'Resource_B', '2018-02-01T00:05:00.000+0000'),
  ('User_C', 'Resource_B', '2018-02-01T00:05:01.000+0000'),
  ('User_C', 'Resource_A', '2018-02-01T00:05:02.000+0000'),
  ('User_C', 'Resource_D', '2018-02-01T00:07:00.000+0000'),
  ('User_D', 'Resource_A', '2018-02-01T00:06:00.000+0000'),
  ('User_D', 'Resource_B', '2018-02-01T00:08:00.000+0000'),
  ('User_D', 'Resource_C', '2018-02-01T00:08:01.000+0000'),
  ('User_D', 'Resource_D', '2018-02-01T00:08:04.000+0000'),
  ('User_D', 'Resource_D', '2018-02-01T00:09:00.000+0000'),
  ('User_D', 'Resource_F', '2018-02-01T00:09:03.000+0000'),
  ('User_D', 'Resource_G', '2018-02-01T00:11:00.000+0000'),
  ('User_D', 'Resource_H', '2018-02-01T00:12:00.000+0000'),
  ('User_D', 'Resource_I', '2018-02-01T00:13:00.000+0000'),
  ('User_D', 'Resource_J', '2018-02-01T00:14:00.000+0000'),
  ('User_D', 'Resource_B', '2018-02-01T00:14:01.000+0000'),
  ('User_D', 'Resource_C', '2018-02-01T00:14:03.000+0000'),
  ('User_D', 'Resource_A', '2018-02-01T00:14:08.000+0000'),
  ('User_D', 'Resource_B', '2018-02-01T00:14:09.000+0000'),
  ('User_D', 'Resource_C', '2018-02-01T00:14:40.000+0000'),
  ('User_D', 'Resource_A', '2018-02-01T00:14:50.000+0000'),
  ('User_E', 'Resource_A', '2018-02-01T00:01:00.000+0000'),
  ('User_E', 'Resource_A', '2018-02-01T00:09:00.000+0000'),

]

user_duplicate_resources = process_test_data(user_duplicate_resources)
user_duplicate_resources.show()

+-----------+-----------+-------------------+
|SM_USERNAME|SM_RESOURCE|       SM_TIMESTAMP|
+-----------+-----------+-------------------+
|     User_A| Resource_A|2018-02-01 00:00:00|
|     User_A| Resource_B|2018-02-01 00:01:00|
|     User_A| Resource_B|2018-02-01 00:02:00|
|     User_B| Resource_A|2018-02-01 00:01:00|
|     User_B| Resource_B|2018-02-01 00:02:00|
|     User_B| Resource_C|2018-02-01 00:03:00|
|     User_B| Resource_C|2018-02-01 00:04:00|
|     User_B| Resource_C|2018-02-01 00:05:00|
|     User_B| Resource_B|2018-02-01 00:06:00|
|     User_B| Resource_D|2018-02-01 00:07:00|
|     User_B| Resource_A|2018-02-01 00:08:00|
|     User_B| Resource_E|2018-02-01 00:09:00|
|     User_C| Resource_A|2018-02-01 00:02:00|
|     User_C| Resource_B|2018-02-01 00:05:00|
|     User_C| Resource_B|2018-02-01 00:05:01|
|     User_C| Resource_A|2018-02-01 00:05:02|
|     User_C| Resource_D|2018-02-01 00:07:00|
|     User_D| Resource_A|2018-02-01 00:06:00|
|     User_D| Resource_B|2018-02-0

In [20]:
expected_user_duplicate_resources = [
  
  ('User_A', {"start": "2018-02-01T00:00:00.000+0000", "end": "2018-02-01T00:15:00.000+0000"}, ["Resource_A", "Resource_B", "Resource_B"]),
  ('User_B', {"start": "2018-02-01T00:00:00.000+0000", "end": "2018-02-01T00:15:00.000+0000"}, ["Resource_A", "Resource_B", "Resource_C", "Resource_C", "Resource_C"]),
  ('User_C', {"start": "2018-02-01T00:00:00.000+0000", "end": "2018-02-01T00:15:00.000+0000"}, ["Resource_A", "Resource_B", "Resource_B", "Resource_A", "Resource_D"]),
  ('User_D', {"start": "2018-02-01T00:00:00.000+0000", "end": "2018-02-01T00:15:00.000+0000"}, ["Resource_A", "Resource_B", "Resource_C", "Resource_D", "Resource_D"]),  
  ('User_E', {"start": "2018-02-01T00:00:00.000+0000", "end": "2018-02-01T00:15:00.000+0000"}, ["Resource_A", "Resource_A"]),
]

expected_user_duplicate_resources = process_expected_data(expected_user_duplicate_resources)
expected_user_duplicate_resources.show()

+-----------+--------------------+--------------------+
|SM_USERNAME|              window|         SM_RESOURCE|
+-----------+--------------------+--------------------+
|     User_A|{2018-02-01 00:00...|[Resource_A, Reso...|
|     User_B|{2018-02-01 00:00...|[Resource_A, Reso...|
|     User_C|{2018-02-01 00:00...|[Resource_A, Reso...|
|     User_D|{2018-02-01 00:00...|[Resource_A, Reso...|
|     User_E|{2018-02-01 00:00...|[Resource_A, Reso...|
+-----------+--------------------+--------------------+



In [21]:
user_duplicate_rows = [
  
  ('User_A', 'Resource_A', '2018-02-01T00:00:00.000+0000'),
  ('User_A', 'Resource_B', '2018-02-01T00:01:00.000+0000'),
  ('User_A', 'Resource_B', '2018-02-01T00:01:00.000+0000'),
  ('User_B', 'Resource_A', '2018-02-01T00:01:00.000+0000'),
  ('User_B', 'Resource_B', '2018-02-01T00:02:00.000+0000'),
  ('User_B', 'Resource_C', '2018-02-01T00:03:00.000+0000'),
  ('User_B', 'Resource_C', '2018-02-01T00:03:00.000+0000'),
  ('User_B', 'Resource_C', '2018-02-01T00:03:00.000+0000'),
  ('User_B', 'Resource_B', '2018-02-01T00:06:00.000+0000'),
  ('User_B', 'Resource_D', '2018-02-01T00:07:00.000+0000'),
  ('User_B', 'Resource_A', '2018-02-01T00:08:00.000+0000'),
  ('User_B', 'Resource_E', '2018-02-01T00:09:00.000+0000'),
  ('User_C', 'Resource_A', '2018-02-01T00:02:00.000+0000'),
  ('User_C', 'Resource_B', '2018-02-01T00:05:00.000+0000'),
  ('User_C', 'Resource_B', '2018-02-01T00:05:00.000+0000'),
  ('User_C', 'Resource_A', '2018-02-01T00:05:02.000+0000'),
  ('User_C', 'Resource_D', '2018-02-01T00:07:00.000+0000'),
  ('User_D', 'Resource_A', '2018-02-01T00:06:00.000+0000'),
  ('User_D', 'Resource_B', '2018-02-01T00:08:00.000+0000'),
  ('User_D', 'Resource_C', '2018-02-01T00:08:01.000+0000'),
  ('User_D', 'Resource_D', '2018-02-01T00:08:04.000+0000'),
  ('User_D', 'Resource_D', '2018-02-01T00:09:00.000+0000'),
  ('User_D', 'Resource_F', '2018-02-01T00:09:03.000+0000'),
  ('User_D', 'Resource_G', '2018-02-01T00:11:00.000+0000'),
  ('User_D', 'Resource_H', '2018-02-01T00:12:00.000+0000'),
  ('User_D', 'Resource_I', '2018-02-01T00:13:00.000+0000'),
  ('User_D', 'Resource_J', '2018-02-01T00:14:00.000+0000'),
  ('User_D', 'Resource_B', '2018-02-01T00:14:01.000+0000'),
  ('User_D', 'Resource_C', '2018-02-01T00:14:03.000+0000'),
  ('User_D', 'Resource_A', '2018-02-01T00:06:00.000+0000'),
  ('User_D', 'Resource_A', '2018-02-01T00:06:00.000+0000'),
  ('User_D', 'Resource_A', '2018-02-01T00:06:00.000+0000'),
  ('User_D', 'Resource_A', '2018-02-01T00:06:00.000+0000'),
  ('User_E', 'Resource_A', '2018-02-01T00:01:00.000+0000'),
  ('User_E', 'Resource_A', '2018-02-01T00:01:00.000+0000'),

]

user_duplicate_rows = process_test_data(user_duplicate_rows)
user_duplicate_rows.show()

+-----------+-----------+-------------------+
|SM_USERNAME|SM_RESOURCE|       SM_TIMESTAMP|
+-----------+-----------+-------------------+
|     User_A| Resource_A|2018-02-01 00:00:00|
|     User_A| Resource_B|2018-02-01 00:01:00|
|     User_A| Resource_B|2018-02-01 00:01:00|
|     User_B| Resource_A|2018-02-01 00:01:00|
|     User_B| Resource_B|2018-02-01 00:02:00|
|     User_B| Resource_C|2018-02-01 00:03:00|
|     User_B| Resource_C|2018-02-01 00:03:00|
|     User_B| Resource_C|2018-02-01 00:03:00|
|     User_B| Resource_B|2018-02-01 00:06:00|
|     User_B| Resource_D|2018-02-01 00:07:00|
|     User_B| Resource_A|2018-02-01 00:08:00|
|     User_B| Resource_E|2018-02-01 00:09:00|
|     User_C| Resource_A|2018-02-01 00:02:00|
|     User_C| Resource_B|2018-02-01 00:05:00|
|     User_C| Resource_B|2018-02-01 00:05:00|
|     User_C| Resource_A|2018-02-01 00:05:02|
|     User_C| Resource_D|2018-02-01 00:07:00|
|     User_D| Resource_A|2018-02-01 00:06:00|
|     User_D| Resource_B|2018-02-0

In [22]:
expected_user_duplicate_rows = [
  
  ('User_A', {"start": "2018-02-01T00:00:00.000+0000", "end": "2018-02-01T00:15:00.000+0000"}, ["Resource_A", "Resource_B", "Resource_B"]),
  ('User_B', {"start": "2018-02-01T00:00:00.000+0000", "end": "2018-02-01T00:15:00.000+0000"}, ["Resource_A", "Resource_B", "Resource_C", "Resource_C", "Resource_C", "Resource_B", "Resource_D"]),
  ('User_C', {"start": "2018-02-01T00:00:00.000+0000", "end": "2018-02-01T00:15:00.000+0000"}, ["Resource_A", "Resource_B", "Resource_B", "Resource_A", "Resource_D"]),
  ('User_D', {"start": "2018-02-01T00:00:00.000+0000", "end": "2018-02-01T00:15:00.000+0000"}, ["Resource_A", "Resource_A", "Resource_A", "Resource_A", "Resource_A", "Resource_B", "Resource_C", "Resource_D", "Resource_D"]),  
  ('User_E', {"start": "2018-02-01T00:00:00.000+0000", "end": "2018-02-01T00:15:00.000+0000"}, ["Resource_A", "Resource_A"]),
]

expected_user_duplicate_rows = process_expected_data(expected_user_duplicate_rows)
expected_user_duplicate_rows.show()

+-----------+--------------------+--------------------+
|SM_USERNAME|              window|         SM_RESOURCE|
+-----------+--------------------+--------------------+
|     User_A|{2018-02-01 00:00...|[Resource_A, Reso...|
|     User_B|{2018-02-01 00:00...|[Resource_A, Reso...|
|     User_C|{2018-02-01 00:00...|[Resource_A, Reso...|
|     User_D|{2018-02-01 00:00...|[Resource_A, Reso...|
|     User_E|{2018-02-01 00:00...|[Resource_A, Reso...|
+-----------+--------------------+--------------------+



In [23]:
user_based_grouping = [
  
  ('User_A', 'Resource_A', '2018-02-01T00:00:00.000+0000'),
  ('User_A', 'Resource_B', '2018-02-01T00:01:00.000+0000'),
  ('User_A', 'Resource_C', '2018-02-01T00:02:00.000+0000'),
  ('User_B', 'Resource_A', '2018-02-01T00:00:00.000+0000'),
  ('User_B', 'Resource_B', '2018-02-01T00:01:00.000+0000'),
  ('User_B', 'Resource_C', '2018-02-01T00:02:00.000+0000'),
  ('User_B', 'Resource_D', '2018-02-01T00:04:00.000+0000'),
  ('User_B', 'Resource_E', '2018-02-01T00:05:00.000+0000'),
  ('User_B', 'Resource_B', '2018-02-01T00:06:00.000+0000'),
  ('User_B', 'Resource_D', '2018-02-01T00:07:00.000+0000'),
  ('User_B', 'Resource_A', '2018-02-01T00:08:00.000+0000'),
  ('User_B', 'Resource_E', '2018-02-01T00:09:00.000+0000'),
  ('User_C', 'Resource_A', '2018-02-01T00:00:00.000+0000'),
  ('User_C', 'Resource_B', '2018-02-01T00:01:00.000+0000'),
  ('User_C', 'Resource_C', '2018-02-01T00:02:01.000+0000'),
  ('User_C', 'Resource_A', '2018-02-01T00:05:02.000+0000'),
  ('User_C', 'Resource_D', '2018-02-01T00:07:00.000+0000'),
  ('User_D', 'Resource_A', '2018-02-01T00:00:00.000+0000'),
  ('User_D', 'Resource_B', '2018-02-01T00:01:00.000+0000'),
  ('User_D', 'Resource_C', '2018-02-01T00:02:01.000+0000'),
  ('User_D', 'Resource_D', '2018-02-01T00:08:04.000+0000'),
  ('User_D', 'Resource_E', '2018-02-01T00:09:00.000+0000'),
  ('User_D', 'Resource_F', '2018-02-01T00:09:03.000+0000'),
  ('User_D', 'Resource_G', '2018-02-01T00:11:00.000+0000'),
  ('User_D', 'Resource_H', '2018-02-01T00:12:00.000+0000'),
  ('User_D', 'Resource_I', '2018-02-01T00:13:00.000+0000'),
  ('User_D', 'Resource_J', '2018-02-01T00:14:00.000+0000'),
  ('User_D', 'Resource_B', '2018-02-01T00:14:01.000+0000'),
  ('User_D', 'Resource_C', '2018-02-01T00:14:03.000+0000'),
  ('User_D', 'Resource_A', '2018-02-01T00:14:08.000+0000'),
  ('User_D', 'Resource_B', '2018-02-01T00:14:09.000+0000'),
  ('User_D', 'Resource_C', '2018-02-01T00:14:40.000+0000'),
  ('User_D', 'Resource_A', '2018-02-01T00:14:50.000+0000'),
  ('User_E', 'Resource_A', '2018-02-01T00:00:00.000+0000'),
  ('User_E', 'Resource_B', '2018-02-01T00:01:00.000+0000'),

]

user_based_grouping = process_test_data(user_based_grouping)
user_based_grouping.show()

+-----------+-----------+-------------------+
|SM_USERNAME|SM_RESOURCE|       SM_TIMESTAMP|
+-----------+-----------+-------------------+
|     User_A| Resource_A|2018-02-01 00:00:00|
|     User_A| Resource_B|2018-02-01 00:01:00|
|     User_A| Resource_C|2018-02-01 00:02:00|
|     User_B| Resource_A|2018-02-01 00:00:00|
|     User_B| Resource_B|2018-02-01 00:01:00|
|     User_B| Resource_C|2018-02-01 00:02:00|
|     User_B| Resource_D|2018-02-01 00:04:00|
|     User_B| Resource_E|2018-02-01 00:05:00|
|     User_B| Resource_B|2018-02-01 00:06:00|
|     User_B| Resource_D|2018-02-01 00:07:00|
|     User_B| Resource_A|2018-02-01 00:08:00|
|     User_B| Resource_E|2018-02-01 00:09:00|
|     User_C| Resource_A|2018-02-01 00:00:00|
|     User_C| Resource_B|2018-02-01 00:01:00|
|     User_C| Resource_C|2018-02-01 00:02:01|
|     User_C| Resource_A|2018-02-01 00:05:02|
|     User_C| Resource_D|2018-02-01 00:07:00|
|     User_D| Resource_A|2018-02-01 00:00:00|
|     User_D| Resource_B|2018-02-0

In [24]:
expected_user_based_grouping = [
  
  ('User_A', {"start": "2018-02-01T00:00:00.000+0000", "end": "2018-02-01T00:15:00.000+0000"}, ["Resource_A", "Resource_B", "Resource_C"]),
  ('User_B', {"start": "2018-02-01T00:00:00.000+0000", "end": "2018-02-01T00:15:00.000+0000"}, ["Resource_A", "Resource_B", "Resource_C", "Resource_D", "Resource_E"]),
  ('User_C', {"start": "2018-02-01T00:00:00.000+0000", "end": "2018-02-01T00:15:00.000+0000"}, ["Resource_A", "Resource_B", "Resource_C", "Resource_A", "Resource_D"]),
  ('User_D', {"start": "2018-02-01T00:00:00.000+0000", "end": "2018-02-01T00:15:00.000+0000"}, ["Resource_A", "Resource_B", "Resource_C", "Resource_D", "Resource_E"]),  
  ('User_E', {"start": "2018-02-01T00:00:00.000+0000", "end": "2018-02-01T00:15:00.000+0000"}, ["Resource_A", "Resource_B"]),
]

expected_user_based_grouping = process_expected_data(expected_user_based_grouping)
expected_user_based_grouping.show()

+-----------+--------------------+--------------------+
|SM_USERNAME|              window|         SM_RESOURCE|
+-----------+--------------------+--------------------+
|     User_A|{2018-02-01 00:00...|[Resource_A, Reso...|
|     User_B|{2018-02-01 00:00...|[Resource_A, Reso...|
|     User_C|{2018-02-01 00:00...|[Resource_A, Reso...|
|     User_D|{2018-02-01 00:00...|[Resource_A, Reso...|
|     User_E|{2018-02-01 00:00...|[Resource_A, Reso...|
+-----------+--------------------+--------------------+



In [25]:
user_shuffled_data = [
  
  ('User_A', 'Resource_A', '2018-02-01T00:10:00.000+0000'),
  ('User_B', 'Resource_A', '2018-02-01T00:10:00.000+0000'),
  ('User_B', 'Resource_B', '2018-02-01T00:02:00.000+0000'),
  ('User_A', 'Resource_B', '2018-02-01T00:01:00.000+0000'),
  ('User_A', 'Resource_C', '2018-02-01T00:02:00.000+0000'),
  ('User_C', 'Resource_A', '2018-02-01T00:10:00.000+0000'),
  ('User_C', 'Resource_C', '2018-02-01T00:05:00.000+0000'),
  ('User_B', 'Resource_C', '2018-02-01T00:03:00.000+0000'),
  ('User_B', 'Resource_D', '2018-02-01T00:04:00.000+0000'),
  ('User_B', 'Resource_E', '2018-02-01T00:05:00.000+0000'),
  ('User_B', 'Resource_B', '2018-02-01T00:06:00.000+0000'),
  ('User_B', 'Resource_D', '2018-02-01T00:07:00.000+0000'),
  ('User_B', 'Resource_A', '2018-02-01T00:08:00.000+0000'),
  ('User_B', 'Resource_E', '2018-02-01T00:09:00.000+0000'),
  ('User_C', 'Resource_B', '2018-02-01T00:05:01.000+0000'),
  ('User_C', 'Resource_A', '2018-02-01T00:05:02.000+0000'),
  ('User_C', 'Resource_D', '2018-02-01T00:07:00.000+0000'),
  ('User_D', 'Resource_A', '2018-02-01T00:10:00.000+0000'),
  ('User_D', 'Resource_B', '2018-02-01T00:08:00.000+0000'),
  ('User_D', 'Resource_C', '2018-02-01T00:08:01.000+0000'),
  ('User_D', 'Resource_D', '2018-02-01T00:08:04.000+0000'),
  ('User_D', 'Resource_E', '2018-02-01T00:09:00.000+0000'),
  ('User_D', 'Resource_F', '2018-02-01T00:09:03.000+0000'),
  ('User_D', 'Resource_G', '2018-02-01T00:11:00.000+0000'),
  ('User_D', 'Resource_H', '2018-02-01T00:12:00.000+0000'),
  ('User_D', 'Resource_I', '2018-02-01T00:13:00.000+0000'),
  ('User_D', 'Resource_J', '2018-02-01T00:14:00.000+0000'),
  ('User_D', 'Resource_B', '2018-02-01T00:14:01.000+0000'),
  ('User_D', 'Resource_C', '2018-02-01T00:14:03.000+0000'),
  ('User_D', 'Resource_A', '2018-02-01T00:14:08.000+0000'),
  ('User_D', 'Resource_B', '2018-02-01T00:14:09.000+0000'),
  ('User_D', 'Resource_C', '2018-02-01T00:14:40.000+0000'),
  ('User_D', 'Resource_A', '2018-02-01T00:14:50.000+0000'),
  ('User_E', 'Resource_A', '2018-02-01T00:10:00.000+0000'),
  ('User_E', 'Resource_C', '2018-02-01T00:09:00.000+0000'),

]

user_shuffled_data = process_test_data(user_shuffled_data)
user_shuffled_data.show()

+-----------+-----------+-------------------+
|SM_USERNAME|SM_RESOURCE|       SM_TIMESTAMP|
+-----------+-----------+-------------------+
|     User_A| Resource_A|2018-02-01 00:10:00|
|     User_B| Resource_A|2018-02-01 00:10:00|
|     User_B| Resource_B|2018-02-01 00:02:00|
|     User_A| Resource_B|2018-02-01 00:01:00|
|     User_A| Resource_C|2018-02-01 00:02:00|
|     User_C| Resource_A|2018-02-01 00:10:00|
|     User_C| Resource_C|2018-02-01 00:05:00|
|     User_B| Resource_C|2018-02-01 00:03:00|
|     User_B| Resource_D|2018-02-01 00:04:00|
|     User_B| Resource_E|2018-02-01 00:05:00|
|     User_B| Resource_B|2018-02-01 00:06:00|
|     User_B| Resource_D|2018-02-01 00:07:00|
|     User_B| Resource_A|2018-02-01 00:08:00|
|     User_B| Resource_E|2018-02-01 00:09:00|
|     User_C| Resource_B|2018-02-01 00:05:01|
|     User_C| Resource_A|2018-02-01 00:05:02|
|     User_C| Resource_D|2018-02-01 00:07:00|
|     User_D| Resource_A|2018-02-01 00:10:00|
|     User_D| Resource_B|2018-02-0

In [26]:
exppected_user_shuffled_data = [
  
  ('User_A', {"start": "2018-02-01T00:00:00.000+0000", "end": "2018-02-01T00:15:00.000+0000"}, ["Resource_B", "Resource_C", "Resource_A",]),
  ('User_B', {"start": "2018-02-01T00:00:00.000+0000", "end": "2018-02-01T00:15:00.000+0000"}, ["Resource_B", "Resource_C", "Resource_D", "Resource_E", "Resource_B"]),
  ('User_C', {"start": "2018-02-01T00:00:00.000+0000", "end": "2018-02-01T00:15:00.000+0000"}, ["Resource_C", "Resource_B", "Resource_A", "Resource_D", "Resource_A"]),
  ('User_D', {"start": "2018-02-01T00:00:00.000+0000", "end": "2018-02-01T00:15:00.000+0000"}, ["Resource_B", "Resource_C", "Resource_D", "Resource_E", "Resource_F"]),  
  ('User_E', {"start": "2018-02-01T00:00:00.000+0000", "end": "2018-02-01T00:15:00.000+0000"}, ["Resource_C", "Resource_A"]),
]

exppected_user_shuffled_data = process_expected_data(exppected_user_shuffled_data)
exppected_user_shuffled_data.show()

+-----------+--------------------+--------------------+
|SM_USERNAME|              window|         SM_RESOURCE|
+-----------+--------------------+--------------------+
|     User_A|{2018-02-01 00:00...|[Resource_B, Reso...|
|     User_B|{2018-02-01 00:00...|[Resource_B, Reso...|
|     User_C|{2018-02-01 00:00...|[Resource_C, Reso...|
|     User_D|{2018-02-01 00:00...|[Resource_B, Reso...|
|     User_E|{2018-02-01 00:00...|[Resource_C, Reso...|
+-----------+--------------------+--------------------+

